In [2]:
import numpy as np 
import pandas as pd

from typing import List
from IPython.display import display

import nltk
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score
from sklearn.naive_bayes import MultinomialNB

from sklearn.linear_model import LogisticRegression

from sklearn import tree
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier 

In [3]:
def read_data_from_csv(file_path: str) -> pd.DataFrame:
    return pd.read_csv(file_path)

In [ ]:

REVIEWS_DATASET_PATH = './datasets/Womens_Clothing_E-Commerce_Reviews.csv'
reviews_dataset = read_data_from_csv(file_path=REVIEWS_DATASET_PATH)

## Resumen del dataset

El dataset contiene 23486 filas, cada una correspondiente a una reseña con unas 10 columnas correspondiente a variables explicadas a continuación.
### Descripción de cada columna

| Columna  | Descripción  |

| Clothing ID | Identificador numérico de la prenda especificada en la crítica |

| Age | Valor numérico correspondiente a la edad de la persona que realizo la crítica |

| Title | Titulo o encabezado de la crítica |

| Reviw Text | Contenido de la reseña realizada |

| Rating | Valor numérico en el rango del 1 al 5, siendo el 1 representante de máxima inconformidad y el 5 correspondiente al mejor valor posible |

| Recommended IND | Valor entre 0 y 1, representando respectivamente si el cliente no recomienda el producto o por el contrario si lo recomienda |

| Positive Feedback Count | Cantidad de reseñas positivas recibidas para el mismo producto |

| Division Name | Nombre de la división a la cual pertenece el producto |

| Department Name | Nombre del departamento al cual pertenece el producto |

| Class Name | Nombre de la clase al cual pertenece el producto |

### Categoria de variables

- Variables Cuantitativas : Age, Positive Feedback Count.

- Variables Cualitativas : Clothing ID, Title, Review Text, Rating, Recommended IND, Division Name, Department Name, Class Name.


In [ ]:
reviews_dataset.dtypes

In [ ]:
reviews_dataset

## Limpieza de Datos

### DATOS NULOS

Observamos la cantidad de nulos que poseen cada columna y el porcentaje que representan dicha cantidad de datos en cada columna

In [ ]:
reviews_dataset.isna().sum()

In [ ]:
reviews_dataset.isnull().sum() *100 / len(reviews_dataset.index)

Debido a que el trabajo a realizar con el dataset consiste en clasificar una reseña como una crítica positiva o una negativa a partir del texto en el campo “Review Text”, decidimos eliminar las columnas que no resultan de utilidad para dicho objetivo. De esta forma, obtenemos un dataset con las columnas necesarias para identificar las distintas prendas, el texto escrito en la reseña y por último la columna "Rating" para el desarrollo del ítem D, motivo por el cual conservamos dicha columna.

In [ ]:
columns_eliminate= ['Unnamed: 0','Age','Title','Recommended IND','Positive Feedback Count','Division Name','Department Name','Class Name']
reviews_dataset.drop(columns_eliminate,axis = 'columns',inplace = True)

reviews_dataset

Por ultimo dado a que las filas con valores inválidos en la columna de "Review Text" son un total de 845 que representan una cantidad menor al 4% de la totalidad de los datos, optamos por eliminar dichas filas.

In [ ]:
reviews_dataset.dropna(inplace=True)
reviews_dataset.shape

## Construccion de una nueva variable:

## Verificación:
Compruebo que la variable Rating no posea valores fuera de rango (1 al 5) antes de operar con dicha columna

In [ ]:
reviews_dataset.Rating.value_counts()

In [ ]:
reviews_dataset['Rating Category'] = reviews_dataset['Rating'].apply(lambda x: 'Negativo' if x <= 3 else 'Positivo')
reviews_dataset

In [ ]:
reviews = reviews_dataset.loc[:, ['Review Text', 'Rating Category']]
reviews

In [ ]:
le = LabelEncoder()
reviews['Rating Category'] = le.fit_transform(reviews['Rating Category'])
reviews

In [ ]:
reviews.dtypes

In [ ]:
stop_words = set(stopwords.words('english'))
vect = CountVectorizer(stop_words=stop_words)
vect_df = vect.fit_transform(reviews['Review Text'].values.tolist())
vect_df.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(vect_df, reviews['Rating Category'], test_size=0.3, random_state=10)

### Naive Bayes

In [ ]:
nb_clf = MultinomialNB()
nb_clf.fit(X_train, y_train)

In [ ]:
y_nb_pred = nb_clf.predict(X_test)
print(classification_report(y_test, y_nb_pred, digits=3))

### Regresión Logística

Para poder utilizar l función, Fit se deben verificar dos cosas:

i. Que no haya datos NaN

ii. Que cada elemento en la fila X_train, sea un vector.

In [ ]:
lr = LogisticRegression(C=0.5, max_iter=150)
lr.fit(X_train, y_train)

In [ ]:
y_lr_pred = lr.predict(X_test)
print(classification_report(y_test, y_lr_pred, digits=3))

### Árboles de decisión

Utilizamos Cross Validation para buscar los hiperparámetros del modelo que permitan obtener los mejores resultados. De esta forma, utilizaremos una de las métricas para buscar los valores que den mejores resultados y utilizarlos para crear nuestro modelo. Dada la cantidad de parámetros y el rango de los mismos utilizamos una randomSearch en lugar de una GridSearch, fijando la cantidad de iteraciones en 20.

In [ ]:
params = {'criterion':['gini','entropy'],
               'min_samples_split': list(range(2,25)),
               'min_samples_leaf':list(range(1,10)),
               'ccp_alpha':np.linspace(0,0.05,2),
               'max_depth':list(range(1,15))}

base_tree = tree.DecisionTreeClassifier() 


tree_random_cv = RandomizedSearchCV(estimator=base_tree,param_distributions = params, scoring= "f1", cv= 7, n_iter= 20, random_state = 10) 

tree_random_cv.fit(X_train,y_train)

print(tree_random_cv.best_params_)
print(tree_random_cv.best_score_)

In [ ]:
tree_model = tree_random_cv.best_estimator_

y_tree_pred = tree_model.predict(X_test)

print(classification_report(y_test, y_tree_pred, digits=3))

### Random Forest

Al igual que para un arbol de decicion buscamos los hiperparametros que nos den mejores resultados para el modelo de Random Forest. En el modelo de Random Forest vamos a utilizar cross validation para los hipervalores: 'criterion', 'min_samples_split', 'min_samples_leaf', 'ccp_alpha' y 'max_depth'.

In [ ]:
params = {'criterion':['gini','entropy'],'min_samples_split': list(range(2,25)),'min_samples_leaf':list(range(1,10)),
          'ccp_alpha':np.linspace(0,0.05,2),'max_depth':list(range(1,15)),'n_estimators': [15,30,60] }


forest_model= RandomForestClassifier() 

r_forest_random_cv = RandomizedSearchCV(estimator=forest_model,param_distributions = params, scoring= 'f1', cv= 7, 
                                      n_iter= 20, random_state = 10) 

r_forest_random_cv.fit(X_train,y_train)

print(r_forest_random_cv.best_params_)
print(r_forest_random_cv.best_score_)

## Aclaración:
Se utilizo RandomSearch para realizar el proceso de CrossValidation con el fin de utilizar un rango de valores para los hiperparametros, lo que por el contrario si se utiliza GridSearch se debe dar una cantidad de posibles valores para cada parametro mas reducida.

In [ ]:
best_forest_model = r_forest_random_cv.best_estimator_

y_forest_pred = best_forest_model.predict(X_test)

print(classification_report(y_test, y_forest_pred, digits=3))

### PARTE 2

In [ ]:
HOTEL_BOOKINGS_DATASET_PATH = './datasets/hotel_bookings.csv'
hotel_bookings_df = read_data_from_csv(file_path=HOTEL_BOOKINGS_DATASET_PATH)

In [ ]:
hotel_bookings_df

# Información sobre el dataset

## Atributos

| Variable  | Tipo de Variable  |  Descripción  |
|---|---|---|
| hotel  | Variable Cualitativa - Texto - Nominal  | Nombre del hotel  |
|  is_canceled | Variable Cualitativa - Texto - Nominal  |  Valor que indica si la reserva fue cancelada  |
|  lead_time | Variable Cuantitativa - Número - Discreta  | Cantidad de dias entre la reserva y la llegada al hotel  |
|  arrival_date_year | Variable Cuantitativa - Número - Discreta ? |  Año de arrivo |
| arrival_date_month  | Variable Cualitativa - Texto - Nominal ? |  Mes del arrivo |
| arrival_date_week_number  | Variable Cuantitativa - Número - Discreta ?  | Número de la semana del arrivo  |
|  arrival_date_day_of_month | Variable Cuantitativa - Número - Discreta ? | Día del mes del arrivo  |
|  stays_in_weekend_nights  | Variable Cuantitativa - Número - Discreta  |  Días del fin de semana (Sabado - Domingo) que abarca la reserva |
|  stays_in_week_nigths | Variable Cuantitativa - Número - Discreta  | Días de la semana (Lunes a Viernes) que abarca la reserva  |
|  adults | Variable Cuantitativa - Número - Discreta  | Cantidad de adultos  |
|  children | Variable Cuantitativa - Número - Discreta (En el Dataset aparece como continua)  |  Cantidad de niños. Cabe aclarar que los datos provenientes del dataset tiene tipo float y se debe transformar, ya que una reserva no puede tener 1.3 niños  |
| babies  | Variable Cuantitativa - Número - Discreta  | Cantidad de bebes  |
|  meal | Variable Cualitativa - Texto - Nominal  |  Tipo de comida reservada (Detalles en Categorias) |
|  country | Variable Cualitativa - Texto - Nominal  |  Pais de origen  |
| market_segment  | Variable Cualitativa - Texto - Nominal  | Segmento del mercado asignado (Detalles en Categorias)  |
| distribution_channel  |   |   |
|  is_repeated_guest | Variable Cualitativa - Número - Nominal  | Si el huesped ya se hospedo en el hotel  |
|  previous_cancelations |  Variable Cuantitativa - Número - Discreta |  Cantidad de cancelaciones  |
|  previous_bookings_not_canceled |  Variable Cuantitativa - Número - Discreta |  Cantidad de veces que no cancelo la reserva  |
| reserved_room_type  | Variable Cualitativa - Text - Nominal  | Código del tipo de cuarto reservado  |
| booking_changes  |  Variable Cuantitativa - Número - Discreta |  Cantidad de cambios en la reserva desde la fecha de reserva hasta la cancelación o check-in  |
| deposit_types  | Variable Cualitativa - Texto - Discreta  | Tipo de depósito que se hizo (Detalles en Categorias)   |
| agent  | Variable Cualitativa - Número - Nominal  | ID de la agencia que hizo la reserva   |
| company  |  Varaible Cualitativa - Número - Nominal | ID de la compania que hizo la reserva  |
| days_in_waiting_list  |  Variable Cuantitativa - Número - Discreta | Número de días que la reserva estuvo en lista de espera hasta ser confirmada  |
| customer_type  |  Variable Cualitativa - Texto - Nominal |  Tipo de reserva (Detalles en Categorias) |
|  adr |   |   |
|  required_car_parking_spaces |  Variable Cuantitativa - Número - Discreta | Cantidad de espacios de estacionamiento para el huesped  |
| total_of_special_requests  |  Variable Cuantitativa - Número - Discreta | Cantidad de pedidos especiales que hizo el huesped  |
|  reservation_status |  Variable Cualitativa - Texto - Nominal | Estado de la reserva (Detalles en Categorias)  |
|  reservation_status_date | Variable Cuantitativa - Número - Continua  | Fecha de la ultima vez que se actualizo reservation_status  |


## Categorias
- Meal:
    - Undefined/SC : No meal package.
    - BB : Bed & Breakfast.
    - HB : Breakfast and one other meal.
    - FB : Breafast, lunch and dinner.
- Market Segment:
    - TA : Agencias de viaje.
    - TO : Operador turístico.
- Deposit Type:
    - No Deposit : No se hizo un deposito.
    - Non Refund : Un deposito se hizo para el valor total de la reseva.
    - Refundable: Un deposito se hizo para un valor menor al total de la reserva.
- Customer Type:
    - Contract : Cuando la reserva esta asociada a un contrato.
    - Group : Cuando la reserva esta asocidada a un grupo.
    - Transient : Cuando la reserva no esta asociada a un contrato ni a un grupo.
    - Transient-party : Cuando la reserva es del tipo transient pero esta asociada a otra reserva transient.
- Reservation Status:
    - Canceled : La reserva fue cancelada.
    - Check-Out : El cliente hizo el check-in y tambien el check-out.
    - No-Show : El cliente no hizo el check-in pero tampoco cancelo la reserva.

### Analizamos la frecuencia de cada categoria

In [ ]:
def group_by_and_count_category(df: pd.DataFrame, column: str) -> pd.DataFrame:
    group_df = df.groupby([column])[column].count().reset_index(name='count')
    return group_df

def display_count_of_categories(df: pd.DataFrame, list_of_columns: List[str]):
    for categorical_column in list_of_columns:
        display(group_by_and_count_category(df=df, column=categorical_column))

In [ ]:
list_of_category_columns = ['is_canceled', 'meal', 'market_segment', 'reserved_room_type', 'deposit_type', 'customer_type']
display_count_of_categories(df=hotel_bookings_df, list_of_columns=list_of_category_columns)

### Analizamos la cantidad de registros vacios

In [ ]:
display(hotel_bookings_df.isnull().sum())

In [ ]:
def calculate_and_display_percentage_of_null_values(df: pd.DataFrame):
    number_of_rows = len(df)
    count_of_nulls: pd.Series = df.isnull().sum()

    for index, count in count_of_nulls.iteritems():
        if count != 0:
            print('{} percentage of nulls: {} - Total of nulls: {}'.format(index, (count / number_of_rows) * 100, count ))

In [ ]:
calculate_and_display_percentage_of_null_values(hotel_bookings_df)

Analizando la cantidad de nulos que tiene cada una de las columnas, podemos distinguir que la cantidad de nulos de las columnas 'children' y 'country' son estadisticamente despreciable. A estas filas nulas podemos eliminarlas, ya que perder un 0.4 % de la data no nos va a hacer perder demasiada información. A diferencia de las columnas mencionadas anteriormente, las columnas 'agent' y 'company' tienen un porcentaje más alto de nulos que no podemos ignorar. Vamos a seguir analizando la correlación que tienen ambas columnas con la columna a predecir (en este caso 'is_canceled') para observar que tanto peso tiene en nuestro modelo.

## Análisis de correlación

In [ ]:
correlation_df = hotel_bookings_df[['is_canceled', 'agent', 'company']].corr()
sns.heatmap(correlation_df, linewidths=0.5, annot=True)

Como se ve en el Heathmap, la correlación que hay entre 'agent' y 'company' con 'is_canceled' es casi nula. Entonces, ya que estas columnas no tienen una correlación significante con la columna a predecir, decidimos dropearlas

## Variables con más correlación con 'is_canceled'

In [ ]:
def get_biggest_correlations_for_column(df: pd.DataFrame, column: str, max_elements: int) -> pd.DataFrame:
    corr_df = df.corr()[column].sort_values(ascending=False)
    corr_df = corr_df.iloc[1:max_elements]
    corr_df.name = f'{column}_corr'

    return corr_df

In [ ]:
correlation_series = get_biggest_correlations_for_column(df=hotel_bookings_df, column='is_canceled', max_elements=5)
correlation_series

In [ ]:
def display_scatter_plot(df: pd.DataFrame, x_column : str, y_column: str):
    df.plot.scatter(x=x_column, y=y_column)

In [ ]:
for index, _ in correlation_series.iteritems():
    display_scatter_plot(df=hotel_bookings_df, x_column=index, y_column='is_canceled')